<a href="https://colab.research.google.com/github/mukul98m/HR-Analytics/blob/master/mask_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

#from tensorflowf.keras.preprocessing.image.load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
#initialize the initial learning rate, number of epochs to train for and batch size
INIT_LR=1e-4
EPOCHS = 20
BS=32

DIRECTORY = "/content/drive/MyDrive/Dataset/mask_detection/Just_mask_detection/Train"
CATEGORIES = ['WithMask','WithoutMask']

In [ ]:
#GRAB THE LIST OF IMAGES IN OUT DATASET DIRECTORY, THEN INITIALIZE THE LIST OF DATA (I.E., IMAGES) AND CLASS IMAGES

data = []
labels = []

for category in CATEGORIES: # with or iwthout
    path= os.path.join(DIRECTORY,category) #directory with mask
    for img in os.listdir(path):
        img_path=os.path.join(path,img) # direct mask with images
        image=load_img(img_path,target_size=(224,224))
        image=img_to_array(image)
        image=preprocess_input(image)
        
        data.append(image)
        labels.append(category)
        len(data)

In [ ]:
len(data)

10000

In [ ]:
#perform one hot encoding on the labels

lb=LabelBinarizer()
labels=lb.fit_transform(labels)
labels=to_categorical(labels)

data= np.array(data, dtype='float32')
labels=np.array(labels)

(trainX, testX, trainY, testY) = train_test_split(data,labels,test_size=0.2,stratify=labels, random_state=42) 

In [ ]:
# data=np.array(data)
# labels=np.array(labels)

In [ ]:
# Save test and train data for later use
np.save('/content/drive/MyDrive/Dataset/mask_detection/Just_mask_detection/Train/data',data)
np.save('/content/drive/MyDrive/Dataset/mask_detection/Just_mask_detection/Train/labels',labels)



In [ ]:
2+2

4

In [ ]:
# Load saved data
data=np.load('/content/drive/MyDrive/Dataset/mask_detection/Just_mask_detection/Train/data')
labels=np.load'/content/drive/MyDrive/Dataset/mask_detection/Just_mask_detection/Train/labels')


In [ ]:
#construct the training image fenerator for data augumentation

aug = ImageDataGenerator(rotation_range=20,
                         zoom_range=0.15,
                         width_shift_range=0.2,
                         height_shift_range=3.2,
                         shear_range=0.15,
                         horizontal_flip=True,
        
                 fill_mode='nearest')

In [ ]:

#load the MobileNetV2 network, ensuring the head FC layer sets are left off
baseModel = MobileNetV2(weights='imagenet', include_top = False,
                        input_tensor=Input(shape=(224,224,3)))

9412608/9406464 [==============================] - 0s 0us/step


In [ ]:
#construct the head of the model that will be placed on top of the base model

headModel=baseModel.output
headModel=AveragePooling2D(pool_size=(7,7))(headModel)
headModel=Flatten(name='flatten')(headModel)
headModel  = Dense(128,activation='relu')(headModel)
headModel = Dropout(8,5)(headModel)
headModel = Dense(2, activation='softmax')(headModel)

In [ ]:
len(train)

In [ ]:
#place the head FC model on top of the base model and freeze them so they will not be updated during the first training process
model = Model(inputs=baseModel.input, outputs=headModel)


In [ ]:
#loop over all layers in the base model and freeze them so they will nt be 
#updated during the 1st training process
for layer in baseModel.layers:
    layer.trainable= False
    
#compile the head of the netowrk 
opt= Adam(lr=INIT_LR, decay = INIT_LR / EPOCHS)
model.compile(loss='binary_crossentropy',optimizer=opt, metrics=['accuracy'])


H= model.fit(aug.flow(trainX,trainY,batch_size=BS),
             steps_per_epoch=len(trainX)//BS,
             validation_data=(testX,testY),
             validation_steps=len(testX) // BS)

In [ ]:
#make prediction on the testing set

predIdxs = model.predict(testX, batch_size=BS)

In [ ]:
#for each image in the testing set we need to find the index of the lable with corresponding largest predicted probability
predIdxs=np.argmax(predIdxs,axis=1)

In [ ]:
#show nicely  formatted classification report

print(classification_report(testY.argmax(axis=1),predIdxs,target_names==lb.classes_))

In [ ]:
#serialize the model to disk

model.save("mask_detection-1.model",save_format='h5')